<a href="https://colab.research.google.com/github/692-Team-1-NLP/proj-2/blob/main/proj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# edit test- Addi 
#Author: Luke+Veronica
#Description: Functions for retrieving and cleaning corpus
import urllib.request, re

# this function accepts a book title as a parameter and fetches the index based on the title 
def get_index(title):
  last_reg=re.compile(r"\w+$")
  last_word=re.findall(last_reg,title)[0]
  if last_word =="Links":
    #"The Murder on the Links"
    index=1
  elif last_word=="Styles":
    #"The Mysterious Affair at Styles"
    index=2
  elif last_word=="Adversary":
    #"The Secret Adversary"
    index=3
  elif last_word=="Suit":
    #"The Man in the Brown Suit"
    index=4
  elif last_word=="Chimneys":
    #"The Secret of Chimneys"
    index=5
  return index

# this function accepts a book index as a parameter and fetches the title based on the index 
def get_title(index):
  if index==1:
    title='The Murder on the Links'
  elif index==2:
    title='The Mysterious Affair at Styles'
  elif index==3:
    title='The Secret Adversary'
  elif index==4:
    title='The Man in the Brown Suit'  
  elif index==5:
    title='The Secret of Chimneys'
  return title

# this function accepts a book index as a parameter and get the text for the book from project gutenberg 
def get_text(index):
  if index==1:
    #"The Murder on the Links"
    url = "https://www.gutenberg.org/files/58866/58866-0.txt"
  elif index==2:
    #"The Mysterious Affair at Styles"
    url="https://www.gutenberg.org/files/863/863-0.txt"
  elif index==3:
    #"The Secret Adversary"
    url="https://www.gutenberg.org/files/1155/1155-0.txt"
  elif index==4:
    #"The Man in the Brown Suit"
    url="https://www.gutenberg.org/files/61168/61168-0.txt"
  elif index==5:
    #"The Secret of Chimneys"
    url="https://www.gutenberg.org/files/65238/65238-0.txt"
  response = urllib.request.urlopen(url)
  data = response.read()      # a `bytes` object
  text = data.decode('utf-8')
  return text


# this function accepts a book index  and returns an appropriate regex that can carve out chapters for that book
def get_ch_regex(index):
  if index==1:
    ch_carve=re.compile(r'\n\d\d?\s[\'\"\u201c]?[A-Z].*\n')
  elif index==2:
    ch_carve=re.compile(r'CHAPTER\s[IVX]+\.\r\n.*\r\n')
  elif index==3:
    ch_carve=re.compile(r'\r\n\r\n\r\nCHAPTER.*\r\n')
  elif index==4:
    ch_carve=re.compile(r'CHAPTER\s\w+\r\n')  
  elif index==5:
    ch_carve=re.compile(r'\d\d?\r\n\r\n[A-OQ-Z].*\r\n')
  return ch_carve


# this function accepts a book index and chapter contents as parameters and trims out any Project gutenberg related artifacts that are not part of the novel
def trim_contents(ch_contents_dict,index):
  last=len(ch_contents_dict)
  if index==1:
    ch_contents_dict[last]=ch_contents_dict[last].split('\nEnd of Project Gutenberg')[0]
  elif index==2:
    ch_contents_dict[last]=ch_contents_dict[last].split('\nTHE END')[0]
  elif index==3:
    ch_contents_dict[last-1]=ch_contents_dict[last-1].split('\nEnd of the Project Gutenberg')[0]
  elif index==4:
    ch_contents_dict[last-1]=ch_contents_dict[last-1].split('THE END')[0]
  elif index==5:
    ch_contents_dict[last]=re.split(r"TRANSCRIBER",ch_contents_dict[last])[0]
  return ch_contents_dict


# this function accepts chapter as a parameter and removes white spaces
def remove_white(chapter):
  regex=r'[\r\n\u200a_]+'
  chapter = re.sub(regex,' ',chapter)
  return chapter


# this function accepts chapter as a parameter and carves out sentences
def sent_carve(chapter):
  #chapter=re.split(r'(?<![A-H|J-Z])[\.\?!](?![\'\"\u2019\u201a\u201c\u275c\u275f\u201e\u201d\u0022\u275e]\s[a-z])(?![\'\"\u2019\u201a\u201c\u275c\u275f\u201e\u201d\u0022\u275e]\sI said)[\'\"\u2018\u2019\u201c\u201d\)\]]*\s*(?<!\w\.\w)(?<![A-Z][a-z][a-z])(?<![A-Z][a-z])\s+',chapter,flags=re.UNICODE)
  chapter=re.split(r'(?<![^A-Z][A-H|J-Z])(?<!Mr|Ms|Dr)(?<!Mrs)(?<!Mlle)(?<!Melle)(?<!\w\.\w)[\.\?!](?!\.)(?![\'\"\u2019\u201a\u201c\u275c\u275f\u201e\u201d\u0022\u275e]\s[a-z])[\'\"\u2018\u2019\u201c\u201d\)\]]*\s*|\u2014\u201d\s*',chapter,flags=re.UNICODE)
  chapter=chapter[:-1]
  chapter={num+1:contents.lower() for (num,contents) in enumerate(chapter)}
  return chapter


# this function accepts a book title and carves out chapters and returns a dictionary of book title, chapter contents and chapter title
def ch_carve(title):

  index=get_index(title)
  text=get_text(index)
  ch_regex=get_ch_regex(index)
  if index ==3:
    text=re.split("CHAPTER XXVIII.     AND AFTER\r\n\r\n\r\n\r\nPROLOGUE",text)[1]
  if index ==4:
    text=re.split("PROLOGUE",text)[1]
  ch_titles=re.findall(ch_regex,text)
  ch_titles_dict={num+1:remove_white(title.strip()) for (num,title) in enumerate(ch_titles)}
  if index==3 or index ==4:
    ch_titles_dict.update( {0 :"PROLOGUE"} )
  chapters=re.split(ch_regex,text)
  if index==3 or index==4:
    ch_contents_dict = {num:contents for (num,contents) in enumerate(chapters)}  
  elif index ==1 or index ==2 or index==5:
    chapters=chapters[1:]
    ch_contents_dict = {num+1:contents for (num,contents) in enumerate(chapters)}
  ch_contents_dict=trim_contents(ch_contents_dict,index)
  return {"title":title,"contents":ch_contents_dict,"chapters":ch_titles_dict}


# this function calls other helper functions and gets the corpus we will be working with
def get_corpus():
  #tentatively planning to index books from 1 to match chapters
  titles=["The Mysterious Affair at Styles","The Murder on the Links","The Secret Adversary","The Man in the Brown Suit","The Secret of Chimneys"]
  corpus={ get_index(title):ch_carve(title) for title in titles}
  return corpus


# this function calls other helper functions and cleans the corpus
def clean_corpus(corpus):
  for keyb,value in corpus.items():
    for  keyc,value in value["contents"].items():
      corpus[keyb]["contents"][keyc]=sent_carve(remove_white(value))
    
  return corpus


# this function returns each chapter as a single blob of text
def sent_blob(chapter):
  temp=''  
  #chapter=re.split(r'(?<![A-H|J-Z])[\.\?!](?![\'\"\u2019\u201a\u201c\u275c\u275f\u201e\u201d\u0022\u275e]\s[a-z])(?![\'\"\u2019\u201a\u201c\u275c\u275f\u201e\u201d\u0022\u275e]\sI said)[\'\"\u2018\u2019\u201c\u201d\)\]]*\s*(?<!\w\.\w)(?<![A-Z][a-z][a-z])(?<![A-Z][a-z])\s+',chapter,flags=re.UNICODE)
  chapter=re.split(r'(?<![^A-Z][A-H|J-Z])(?<!Mr|Ms|Dr)(?<!Mrs)(?<!Mlle)(?<!Melle)(?<!\w\.\w)[\.\?!](?!\.)(?![\'\"\u2019\u201a\u201c\u275c\u275f\u201e\u201d\u0022\u275e]\s[a-z])[\'\"\u2018\u2019\u201c\u201d\)\]]*\s*|\u2014\u201d\s*',chapter,flags=re.UNICODE)
  chapter=chapter[:-1]
  for ch in chapter:
    temp=temp+" "+ch.lower()+"#"    
  return temp


# this function removes punctuation 
def remove_punc(blob):
  blob=re.sub(r"[\u201c\u201d\?,;:\.!\u2018\u2019\u201a\u275b\u275c\u275f\s-]+" ,' ',blob)
  return blob
def tighten(blob):
  blob= re.sub(r"\s+"," ",blob)
  blob=re.sub(r"#","\n",blob)
  return blob

# this function adds a dictionary to the corpus that contains each book as a single blob of test accessed via corpus["blob"]
def blob_corpus(dirty_corpus):
  for keyb,value in dirty_corpus.items():
    blob=''
    for  keyc,value in value["contents"].items():
      blob=blob+" "+tighten(remove_punc(remove_white(sent_blob(dirty_corpus[keyb]["contents"][keyc]))))
    dirty_corpus[keyb]["blob"]=blob  
  return dirty_corpus

In [2]:
dirty_corpus=get_corpus()
dirty_corpus=blob_corpus(dirty_corpus)
corpus=clean_corpus(dirty_corpus)
big_blob=corpus[1]["blob"]+corpus[2]["blob"]+corpus[3]["blob"]+corpus[4]["blob"]+corpus[5]["blob"]
#print(big_blob)
with open("temp.txt","w") as f:
  f.write(big_blob)

In [3]:
!pip install --upgrade gensim

In [4]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models.word2vec import Word2Vec
class Corpus():
  def __init__(self,path):
    self.path=path
  def __iter__(self):
    corpus_path=datapath(self.path)
    for line in open(corpus_path):
      yield utils.simple_preprocess(remove_stopwords(line))
path='/content/temp.txt'
!pwd
corpus=Corpus(path)
model=Word2Vec(corpus)
model.wv.most_similar('hercule')


/content


[('satisfied', 0.9963435530662537),
 ('figure', 0.9962856769561768),
 ('inclined', 0.9962761998176575),
 ('connection', 0.9962567090988159),
 ('easily', 0.9962537288665771),
 ('lips', 0.9962407350540161),
 ('paused', 0.9962387084960938),
 ('ears', 0.9962373375892639),
 ('somewhat', 0.9962337017059326),
 ('deep', 0.9962244033813477)]

In [5]:

model.wv.most_similar('poirot')


[('tommy', 0.9984163641929626),
 ('virginia', 0.9978669285774231),
 ('julius', 0.997782289981842),
 ('tuppence', 0.9974542856216431),
 ('anthony', 0.9969242215156555),
 ('slowly', 0.996880829334259),
 ('gently', 0.9967830181121826),
 ('thoughtfully', 0.9966556429862976),
 ('surprise', 0.9964278936386108),
 ('smile', 0.9962503910064697)]

In [6]:
#spacy install
!pip install spacy
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 5.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
#spacy code
from spacy.lang.en import English 
import spacy
nlp = spacy.load("en_core_web_sm")
# W.I.P.
import re

dirty_corpus=get_corpus()
all_sentences=''
for bookid in range(1,6):
    nch=len(dirty_corpus[bookid]["chapters"])
    for chid in range(1,nch):
        dirty_chapter=dirty_corpus[bookid]["contents"][chid]
        ch_doc = nlp(dirty_chapter)
        ch_sentences=''
        for sent in ch_doc.sents:
          sent=re.sub(r"\s+",' ',re.sub(r"[\u2000-\u206F!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`\{\|\}~]+",' ',sent.text.strip())).lower().strip()
          if sent!="":
            ch_sentences+=sent+"\n"
    all_sentences+=ch_sentences
print(all_sentences)
with open("temp2.txt","w") as f:
  f.write(all_sentences)





In [19]:
dirty_corpus=get_corpus()
# this function adds a dictionary to the corpus that contains each book as a single blob of test accessed via corpus["blob"]
def sent_blob2(chapter):
  temp=''  
  ch_doc=nlp(chapter)
  for sent in ch_doc.sents:
    sent=re.sub(r"\s+",' ',re.sub(r"[\u2000-\u206F!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`\{\|\}~]+",' ',sent.text.strip())).lower().strip()
    if sent!="":
      temp+=sent+"\n"    
  return temp

def blob_corpus2(dirty_corpus):
  for keyb,value in dirty_corpus.items():
    blob=''
    for  keyc,value in value["contents"].items():
      blob+=sent_blob2(dirty_corpus[keyb]["contents"][keyc])
    dirty_corpus[keyb]["blob"]=blob  
  return dirty_corpus
c2=blob_corpus2(dirty_corpus)
blob2=c2[1]["blob"]+c2[2]["blob"]+c2[3]["blob"]+c2[4]["blob"]+c2[5]["blob"]
with open("temp2.txt","w") as f:
  f.write(blob2)


In [23]:
path='/content/temp2.txt'
!pwd
scapy_corpus=Corpus(path)
sc_model=Word2Vec(scapy_corpus)
sc_model.wv.most_similar('hercule')

/content


[('replied', 0.9969497323036194),
 ('asked', 0.9968485236167908),
 ('sat', 0.9967306852340698),
 ('stared', 0.9966945052146912),
 ('walked', 0.9966557025909424),
 ('surprise', 0.9966446757316589),
 ('continued', 0.9966220855712891),
 ('smiled', 0.9966018795967102),
 ('stepped', 0.9965628981590271),
 ('felt', 0.9965613484382629)]

Test-Addi

In [8]:
print("Test")
print("foo")

Test
foo


This is some text